1. Initialize modules

In [2]:
import os
import sys
import pandas as pd
import numpy as np
from pandas import DataFrame
import pymorphy2
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from joblib import parallel_backend

2. Import files and reading

In [4]:
product = 'data/products.xlsx'
tech = 'data/product_technologies.xlsx'
branch = 'data/product_branch.xlsx'

dfp = pd.read_excel(product, header=1)
dft = pd.read_excel(tech, header=0)
dfb= pd.read_excel(branch, header=0)


dft.rename(columns={ dft.columns[1]: "id_products"}, inplace=True)
dfb.rename(columns={ dfb.columns[1]: "id_products"}, inplace=True)
# Преведение к одному типу данных
# dft['id_products']=dft['id_products'].astype (int)
merge_tech = pd.merge(dft, dfp, how="left", on=["id_products"])
merge_branch = pd.merge(merge_tech, dfb, how="left", on=["id_products"])
products = merge_branch.drop('global_id_x', axis=1)
products = products.drop('global_id_y', axis=1)
products.rename(columns={products.columns[1]: "tech_lvl_1",products.columns[2]: "tech_lvl_2",products.columns[3]: "tech_lvl_3",products.columns[5]: "product_market_name", products.columns[6]: "section_OKPD2",products.columns[7]: "class_OKPD2",products.columns[8]: "subclass_OKPD2",products.columns[9]: "group_OKPD2",products.columns[10]: "subgroup_OKPD2",products.columns[11]: "view_OKPD2",products.columns[12]: "category_OKPD2",products.columns[13]: "subcategory_OKPD2",products.columns[14]: "product_description",products.columns[15]: "branch",products.columns[16]: "subbranch"}, inplace=True)



print("Data frame type: " + str(type(dfp)))

Data frame type: <class 'pandas.core.frame.DataFrame'>


3. Overview DS

In [5]:
print('Overview products')
pd.DataFrame.info(products)
products.head()

Overview products
<class 'pandas.core.frame.DataFrame'>
Int64Index: 13443 entries, 0 to 13442
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id_products          13443 non-null  int64 
 1   tech_lvl_1           13443 non-null  object
 2   tech_lvl_2           13443 non-null  object
 3   tech_lvl_3           13443 non-null  object
 4   id_company           13443 non-null  int64 
 5   product_market_name  13443 non-null  object
 6   section_OKPD2        13443 non-null  object
 7   class_OKPD2          13443 non-null  object
 8   subclass_OKPD2       13443 non-null  object
 9   group_OKPD2          13443 non-null  object
 10  subgroup_OKPD2       13443 non-null  object
 11  view_OKPD2           13443 non-null  object
 12  category_OKPD2       13426 non-null  object
 13  subcategory_OKPD2    13218 non-null  object
 14  product_description  13443 non-null  object
 15  branch               13443 non-null

,id_products,tech_lvl_1,tech_lvl_2,tech_lvl_3,id_company,product_market_name,section_OKPD2,class_OKPD2,subclass_OKPD2,group_OKPD2,subgroup_OKPD2,view_OKPD2,category_OKPD2,subcategory_OKPD2,product_description,branch,subbranch
0,830397,Сбор информации об объекте,Методы биоорганической химии,Иммунологические методы,811033,"Онлайн-ресурс “ImmunoCloud” для обработки, хра...",Раздел J. Услуги в области информации и связи,62. Продукты программные и услуги по разработк...,62.0. Продукты программные и услуги по разрабо...,62.01. Продукты программные и услуги по разраб...,"62.01.1. Услуги по проектированию, разработке ...","62.01.11. Услуги по проектированию, разработке...",62.01.11.000. Услуги по проектированию и разра...,Отсутствует,Компания разрабатывает и успешно коммерциализу...,ЗДРАВООХРАНЕНИЕ,Производство биомедицинских клеточных продуктов
1,830397,Сбор информации об объекте,Методы биоорганической химии,Иммунологические методы,811033,"Онлайн-ресурс “ImmunoCloud” для обработки, хра...",Раздел J. Услуги в области информации и связи,62. Продукты программные и услуги по разработк...,62.0. Продукты программные и услуги по разрабо...,62.01. Продукты программные и услуги по разраб...,"62.01.1. Услуги по проектированию, разработке ...","62.01.11. Услуги по проектированию, разработке...",62.01.11.000. Услуги по проектированию и разра...,Отсутствует,Компания разрабатывает и успешно коммерциализу...,ЗДРАВООХРАНЕНИЕ,Производство лекарственных средств
2,830397,Сбор информации об объекте,Методы биоорганической химии,Иммунологические методы,811033,"Онлайн-ресурс “ImmunoCloud” для обработки, хра...",Раздел J. Услуги в области информации и связи,62. Продукты программные и услуги по разработк...,62.0. Продукты программные и услуги по разрабо...,62.01. Продукты программные и услуги по разраб...,"62.01.1. Услуги по проектированию, разработке ...","62.01.11. Услуги по проектированию, разработке...",62.01.11.000. Услуги по проектированию и разра...,Отсутствует,Компания разрабатывает и успешно коммерциализу...,ЗДРАВООХРАНЕНИЕ,Производство медицинских изделий для диагности...
3,812554,Сбор информации об объекте,Методы биоорганической химии,Технологии секвенирования ДНК и РНК,811195,Продукция для молекулярно-генетической идентиф...,Раздел C. Продукция обрабатывающих производств,"21. Средства лекарственные и материалы, примен...","21.2. Препараты лекарственные и материалы, при...","21.20. Препараты лекарственные и материалы, пр...",21.20.2. Препараты лекарственные прочие и мате...,21.20.23. Реагенты диагностические и прочие фа...,21.20.23.110. Реагенты диагностические,21.20.23.111. Препараты диагностические,Технология высокомультиплексной нормирующей ам...,СЕЛЬСКОЕ ХОЗЯЙСТВО (ВКЛЮЧАЯ РЫБОЛОВСТВО),Животноводство
4,812554,Сбор информации об объекте,Методы биоорганической химии,Технологии секвенирования ДНК и РНК,811195,Продукция для молекулярно-генетической идентиф...,Раздел C. Продукция обрабатывающих производств,"21. Средства лекарственные и материалы, примен...","21.2. Препараты лекарственные и материалы, при...","21.20. Препараты лекарственные и материалы, пр...",21.20.2. Препараты лекарственные прочие и мате...,21.20.23. Реагенты диагностические и прочие фа...,21.20.23.110. Реагенты диагностические,21.20.23.111. Препараты диагностические,Технология высокомультиплексной нормирующей ам...,ЗДРАВООХРАНЕНИЕ,Производство медицинских изделий для диагности...


Data overview
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4761 entries, 0 to 4760
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   global_id               4761 non-null   int64 
 1   id_products             4761 non-null   int64 
 2   Технология (1 уровень)  4761 non-null   object
 3   Технология (2 уровень)  4761 non-null   object
 4   Технология (3 уровень)  4761 non-null   object
dtypes: int64(2), object(3)
memory usage: 186.1+ KB


,global_id,id_products,Отрасль,Подотрасль
0,830782,830283,СЕЛЬСКОЕ ХОЗЯЙСТВО (ВКЛЮЧАЯ РЫБОЛОВСТВО),Животноводство
1,830783,830728,СЕЛЬСКОЕ ХОЗЯЙСТВО (ВКЛЮЧАЯ РЫБОЛОВСТВО),Растениеводство
2,830784,812492,СЕЛЬСКОЕ ХОЗЯЙСТВО (ВКЛЮЧАЯ РЫБОЛОВСТВО),Растениеводство
3,830785,830618,СЕЛЬСКОЕ ХОЗЯЙСТВО (ВКЛЮЧАЯ РЫБОЛОВСТВО),Растениеводство
4,830786,830560,СЕЛЬСКОЕ ХОЗЯЙСТВО (ВКЛЮЧАЯ РЫБОЛОВСТВО),Животноводство


In [16]:
raw_data = products.dropna(subset=['product_description'])
morph = pymorphy2.MorphAnalyzer()
lemm_texts_list = []
for text in raw_data['product_description']:
    text_lem = [morph.parse(word)[0].normal_form for word in text.split(' ')]
    if len(text_lem) <= 2:
        lemm_texts_list.append('')
        continue
    lemm_texts_list.append(' '.join(text_lem))
raw_data['product_description'] = lemm_texts_list
raw_data = raw_data[raw_data['product_description'] != '']
raw_data.head()

,id_products,tech_lvl_1,tech_lvl_2,tech_lvl_3,id_company,product_market_name,section_OKPD2,class_OKPD2,subclass_OKPD2,group_OKPD2,subgroup_OKPD2,view_OKPD2,category_OKPD2,subcategory_OKPD2,product_description,branch,subbranch
0,830397,Сбор информации об объекте,Методы биоорганической химии,Иммунологические методы,811033,"Онлайн-ресурс “ImmunoCloud” для обработки, хра...",Раздел J. Услуги в области информации и связи,62. Продукты программные и услуги по разработк...,62.0. Продукты программные и услуги по разрабо...,62.01. Продукты программные и услуги по разраб...,"62.01.1. Услуги по проектированию, разработке ...","62.01.11. Услуги по проектированию, разработке...",62.01.11.000. Услуги по проектированию и разра...,Отсутствует,компания разрабатывать и успешно коммерциализо...,ЗДРАВООХРАНЕНИЕ,Производство биомедицинских клеточных продуктов
1,830397,Сбор информации об объекте,Методы биоорганической химии,Иммунологические методы,811033,"Онлайн-ресурс “ImmunoCloud” для обработки, хра...",Раздел J. Услуги в области информации и связи,62. Продукты программные и услуги по разработк...,62.0. Продукты программные и услуги по разрабо...,62.01. Продукты программные и услуги по разраб...,"62.01.1. Услуги по проектированию, разработке ...","62.01.11. Услуги по проектированию, разработке...",62.01.11.000. Услуги по проектированию и разра...,Отсутствует,компания разрабатывать и успешно коммерциализо...,ЗДРАВООХРАНЕНИЕ,Производство лекарственных средств
2,830397,Сбор информации об объекте,Методы биоорганической химии,Иммунологические методы,811033,"Онлайн-ресурс “ImmunoCloud” для обработки, хра...",Раздел J. Услуги в области информации и связи,62. Продукты программные и услуги по разработк...,62.0. Продукты программные и услуги по разрабо...,62.01. Продукты программные и услуги по разраб...,"62.01.1. Услуги по проектированию, разработке ...","62.01.11. Услуги по проектированию, разработке...",62.01.11.000. Услуги по проектированию и разра...,Отсутствует,компания разрабатывать и успешно коммерциализо...,ЗДРАВООХРАНЕНИЕ,Производство медицинских изделий для диагности...
3,812554,Сбор информации об объекте,Методы биоорганической химии,Технологии секвенирования ДНК и РНК,811195,Продукция для молекулярно-генетической идентиф...,Раздел C. Продукция обрабатывающих производств,"21. Средства лекарственные и материалы, примен...","21.2. Препараты лекарственные и материалы, при...","21.20. Препараты лекарственные и материалы, пр...",21.20.2. Препараты лекарственные прочие и мате...,21.20.23. Реагенты диагностические и прочие фа...,21.20.23.110. Реагенты диагностические,21.20.23.111. Препараты диагностические,технология высокомультиплексный нормировать ам...,СЕЛЬСКОЕ ХОЗЯЙСТВО (ВКЛЮЧАЯ РЫБОЛОВСТВО),Животноводство
4,812554,Сбор информации об объекте,Методы биоорганической химии,Технологии секвенирования ДНК и РНК,811195,Продукция для молекулярно-генетической идентиф...,Раздел C. Продукция обрабатывающих производств,"21. Средства лекарственные и материалы, примен...","21.2. Препараты лекарственные и материалы, при...","21.20. Препараты лекарственные и материалы, пр...",21.20.2. Препараты лекарственные прочие и мате...,21.20.23. Реагенты диагностические и прочие фа...,21.20.23.110. Реагенты диагностические,21.20.23.111. Препараты диагностические,технология высокомультиплексный нормировать ам...,ЗДРАВООХРАНЕНИЕ,Производство медицинских изделий для диагности...


In [4]:
X = raw_data ['product_description']
y = raw_data ['branch']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state= 42, test_size=0.3)

NameError: name 'raw_data' is not defined

In [3]:
logreg = Pipeline([
                ('vect', CountVectorizer(analyzer='char', ngram_range =([2,10]))),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=8,C=1e5, solver='saga',
                                           multi_class='multinomial',
                                           max_iter=1000,
                                           random_state=42)),
])

logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print(classification_report(y_test, y_pred, target_names=themes))
print(f"F1 Score: {f1_score(y_test, y_pred, average='weighted')}")

NameError: name 'X_train' is not defined

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=0]="ооо "нейромед" разрабатывать программный обеспечение, доступный в веб-интерфейс - цифровой платформа neuromark для визуальный программирование искусственный нейронный сеть глубокий обучения. продукт нацелить на упрощение разработка программный обеспечение или продуктов, использовать технология компьютерный зрение и иметь дело с анализ изображений. при помощь разрабатывать продукт любой человек, не владеть знание программирование и data science, мочь самостоятельно обучить модель на детекцию, сегментация или классификация изображений. пользователь смочь загрузить набор собственный изображений, разметить он в веб-интерфейс продукта, настроить основной блок программирование нейронный сеть в визуальный конструкторе, отправить модель на обучение с использование удалённый вычислительный мощностей. после что пользователь получить готовый нейронный сеть, доступный по api.
ключевой конкурентный преимущество являться 

NameError: name 'train_pool' is not defined